# 基于模式匹配的对话机器人实现

## Pattern Match

In [1]:
import random

In [2]:
def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])

In [3]:
is_variable('?happy')

True

In [4]:
is_variable('happy')

False

In [5]:
def pat_match(pattern,saying):
    if is_variable(pattern[0]):
        print(pattern[0])
        return True
    else:
        if pattern[0] != saying[0]: return False
        else:
            return pat_match(pattern[1:],saying[1:])

In [6]:
pat_match('I want ?X'.split(),'I want holiday'.split())

?X


True

In [7]:
pat_match('I want ?X'.split(),'I like holiday'.split())

False

## 获得匹配的变量

In [8]:
def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])

In [9]:
def pat_match(pattern, saying):
    if is_variable(pattern[0]):
        return pattern[0],saying[0]
    else:
        if pattern[0]!=saying[0]: return False
        else:
            return pat_match(pattern[1:],saying[1:])

In [10]:
pat_match('I want ?X'.split(),'I want holiday'.split())

('?X', 'holiday')

In [11]:
pat_match('?Z equals ?Y'.split(),'3+2 equals 2+2'.split())

('?Z', '3+2')

In [12]:
def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])

In [13]:
def pat_match(pattern, saying):
    if not pattern or not saying: return []
    
    if is_variable(pattern[0]):
        return [(pattern[0],saying[0])]+pat_match(pattern[1:],saying[1:])
    else:
        if pattern[0]!=saying[0]: return []
        else:
            return pat_match(pattern[1:],saying[1:])

In [14]:
pat_match('?Z equals ?Y'.split(),'3+2 equals 2+2'.split())

[('?Z', '3+2'), ('?Y', '2+2')]

## 新建两个函数
1.结果转为一个dictionary
2.根据这个dictionary依照自定义的方式替换

In [15]:
def pat_to_dict(patterns):
    return {k : v for k ,v in patterns}

In [16]:
pat_to_dict(pat_match('?Z equals ?Y'.split(),'3+2 equals 2+2'.split()))

{'?Y': '2+2', '?Z': '3+2'}

In [17]:
def subsitite(rule,parsed_rules):
    if not rule: return []
    return [parsed_rules.get(rule[0],rule[0])]+subsitite(rule[1:],parsed_rules)

In [18]:
got_patterns = pat_match('I want ?X'.split(),'I want iphone'.split())

In [19]:
subsitite("What if you mean if you got a ?X".split(),pat_to_dict(got_patterns))

['What', 'if', 'you', 'mean', 'if', 'you', 'got', 'a', 'iphone']

In [20]:
John_pat = pat_match('?P needs ?X'.split(),'John needs resting'.split())

In [21]:
' '.join(subsitite("What if you mean if you got a ?X".split(),pat_to_dict(got_patterns)))

'What if you mean if you got a iphone'

In [22]:
subsitite("Why does ?P need ?X ?".split(),pat_to_dict(John_pat))

['Why', 'does', 'John', 'need', 'resting', '?']

In [23]:
' '.join(subsitite("Why does ?P need ?X ?".split(),pat_to_dict(John_pat)))

'Why does John need resting ?'

In [24]:
defined_patterns={
    "I need ?X":["Image you will get ?X soon","Why do you need ?X ?"],
    "My ?X told me something":["Talk about more about your ?X","How do you think about your ?X ?"]
}

In [25]:
def get_response(saying): 
    
#     1.首先提取出saying对应define_patterns 的key值
#     2.随机选择该key值对应的value
#     3.将对应value值?X用subsitite()替换掉
    
    
    pattern=[]
    for i in defined_patterns:
        k = pat_match(i.split(), saying.split())
        if k:
            response = random.choice(defined_patterns[i])
            return ' '.join(subsitite(response.split(),pat_to_dict(k)))
            
#     遍历怎么找到对应的？==>不为[] 就返回
#     转换为字典

In [26]:
get_response('I need iPhone')

'Why do you need iPhone ?'

In [27]:
def is_pattern_segment(pattern):
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])

In [28]:
is_pattern_segment('?*P')

True

In [29]:
from collections import defaultdict

In [30]:
fail = [True,None]

def pat_match_with_seg(pattern,saying):
    if not pattern or not saying:return []
    
    pat = pattern[0]
    
    if is_variable(pat): #多个变量的情况
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    elif is_pattern_segment(pat):
        match, index = segment_match(pattern, saying)
        return [match] + pat_match_with_seg(pattern[1:], saying[index:])
    elif  pat == saying[0]:
        return pat_match_with_seg(pattern[1:],saying[1:])
    else:
        return fail
    

In [31]:
def segment_match(pattern, saying):
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')
    
    if not rest: return (seg_pat, saying), len(saying)
    
    for i, token in enumerate(saying): #enumerate() 函数用于将一个可遍历的数据对象(如列表、元组或字符串)组合为一个索引序列，同时列出数据和数据下标，一般用在 for 循环当中。
        if rest[0] == token and is_match(rest[1:], saying[(i + 1):]):
            return(seg_pat, saying[:i]), i
    return (seg_pat, saying), len(saying)

def is_match(rest, saying):
    if not rest and not saying:
        return True
    if not all(a.isalpha() for a in rest[0]):
        return True
    if rest[0]!= saying[0]:
        return False
    return is_match(rest[1:], saying[1:])

In [32]:
segment_match('?*P is very good'.split(),'My dog and my cat is very good'.split())

(('?P', ['My', 'dog', 'and', 'my', 'cat']), 5)

In [33]:
segment_match('?*P is very good and ?*X'.split(),'My dog is very good and my cat is very cute'.split())

(('?P', ['My', 'dog']), 2)

In [34]:
pat_match_with_seg('?*P is very good and ?*X'.split(),'My dog is very good and my cat is very cute'.split())

[('?P', ['My', 'dog']), ('?X', ['my', 'cat', 'is', 'very', 'cute'])]

In [35]:
response_pair = {
    'I need ?X':[
        'Why do you need ?X'
    ],
    "I don't like my ?X":[
        'What bad things did ?X do for you?'
    ]
}

In [36]:
pat_match_with_seg('I need ?*X'.split(),'I need an iPhone'.split())

[('?X', ['an', 'iPhone'])]

In [37]:
subsitite("Why do you need ?X".split(),pat_to_dict(pat_match_with_seg('I need ?*X'.split(),'I need an iPhone'.split())))

['Why', 'do', 'you', 'need', ['an', 'iPhone']]

In [38]:
def pat_to_dict(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in patterns}

In [39]:
subsitite("Why do you need ?X".split(),pat_to_dict(pat_match_with_seg('I need ?*X'.split(),'I need an iPhone'.split())))

['Why', 'do', 'you', 'need', 'an iPhone']

In [40]:
("?*X hello ?*Y","Hi, how do you do")

('?*X hello ?*Y', 'Hi, how do you do')

In [41]:
subsitite("Hi, how do you do".split(),pat_to_dict(pat_match_with_seg('?*X  hello ?*Y'.split(),'I am mike, hello'.split())))

['Hi,', 'how', 'do', 'you', 'do']

In [42]:
rules = {
    '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    "I was ?*X": ["Were you really ?X ?", "I already knew you were ?X ."],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y', 'Do you wish that ?y', 'What do you think about ?y', 'Really-- if ?y'],
    '?*x no ?*y': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?']
}

In [43]:
def get_response2(saying, response_rules): 

    pattern=[]
    for i in response_rules:
        m=segment_match(i.split(),saying.split())
        if m[1]==1:
            k = pat_match_with_seg(i.split(), saying.split()) 
            response = random.choice(response_rules[i])
            return ' '.join(subsitite(response.split(),pat_to_dict(k)))

In [44]:
get_response2('I was ysy ', rules)

'Were you really ysy ?'

### 问题1

In [45]:
def get_response(saying): 
    
#     1.首先提取出saying对应define_patterns 的key值
#     2.随机选择该key值对应的value
#     3.将对应value值?X用subsitite()替换掉
    
    
    pattern=[]
    for i in defined_patterns:
        k = pat_match(i.split(), saying.split())
        if k:
            response = random.choice(defined_patterns[i])
            return ' '.join(subsitite(response.split(),pat_to_dict(k)))
            
#     遍历怎么找到对应的？==>不为[] 就返回
#     转换为字典

In [46]:
get_response('I need iPhone')

'Image you will get iPhone soon'

In [47]:
def get_response2(saying, response_rules): 

    pattern=[]
    for i in response_rules:
        m=segment_match(i.split(),saying.split())
        if m[1]==1:
            k = pat_match_with_seg(i.split(), saying.split()) 
            response = random.choice(response_rules[i])
            return ' '.join(subsitite(response.split(),pat_to_dict(k)))

In [48]:
get_response2('I was a baby ', rules) #为什么一直是'Hi, how do you do?'

'Were you really a baby ?'

### 问题2
改写以上程序，将程序变成能够支持中文输入的模式。 提示: 你可以需用用到 jieba 分词

### 问题3
多设计一些模式，让这个程序变得更好玩，多和大家交流，看看大家有什么好玩的模式

### 问题4
1. 这样的程序有什么优点？有什么缺点？你有什么可以改进的方法吗？
2. 什么是数据驱动？数据驱动在这个程序里如何体现？
3. 数据驱动与 AI 的关系是什么？

1、这样的程序有什么优点/缺点？改进方法？
答： 优点：可以实现一些简单的对话，有规律性的对话
        缺点：需要自定义很多句子，只能根据用户说的话提取一些信息，再根据这个信息做出可能的回答，应用在现实生活中，可能多句话的上下文关系会不紧密。
        改进方法：在程序中加入一些记忆的功能，当处理很多句话的时候，上下文能够有联系，在多种联系的情况下再选择最有可能的回答，我觉得这样的回答会更有逻辑性。

2、什么是数据驱动？数据驱动在这个程序中如何体现？
答：数据驱动是一种方法，可以解决一类问题，而不是具体的一个问题，对于一些近似的或者没有见过的问题也能解决。在本程序中，设计对话的时候，我们并不知道用户会具体说一些什么词，但是他们都有一定的格式，比如我是XXX，我们只需要提取出XXX，加入对话中就可以解决这一类的问题。

3、数据驱动与AI的关系
答：数据驱动可以帮助我们解决一些有规律但是并不完全具体的问题，可以根据很多数据提取出其中的规律，设计出一些patterns用来解决更多的问题。人工智能可以对一些未知的事情自己做出反应，数据驱动可以帮助解决这个问题。

In [49]:
rule_responses = {
    '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y', 'Do you wish that ?y', 'What do you think about ?y', 'Really-- if ?y'],
    '?*x no ?*y': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '?*x I was ?*y': ['Were you really', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x I feel ?*y': ['Do you often feel ?y ?', 'What other feelings do you have?'],
    '?*x你好?*y': ['你好呀', '请告诉我你的问题'],
    '?*x我想?*y': ['你觉得?y有什么意义呢？', '为什么你想?y', '你可以想想你很快就可以?y了'],
    '?*x我想要?*y': ['?x想问你，你觉得?y有什么意义呢?', '为什么你想?y', '?x觉得... 你可以想想你很快就可以有?y了', '你看?x像?y不', '我看你就像?y'],
    '?*x喜欢?*y': ['喜欢?y的哪里？', '?y有什么好的呢？', '你想要?y吗？'],
    '?*x讨厌?*y': ['?y怎么会那么讨厌呢?', '讨厌?y的哪里？', '?y有什么不好呢？', '你不想要?y吗？'],
    '?*xAI?*y': ['你为什么要提AI的事情？', '你为什么觉得AI要解决你的问题？'],
    '?*x机器人?*y': ['你为什么要提机器人的事情？', '你为什么觉得机器人要解决你的问题？'],
    '?*x对不起?*y': ['不用道歉', '你为什么觉得你需要道歉呢?'],
    '?*x我记得?*y': ['你经常会想起这个吗？', '除了?y你还会想起什么吗？', '你为什么和我提起?y'],
    '?*x如果?*y': ['你真的觉得?y会发生吗？', '你希望?y吗?', '真的吗？如果?y的话', '关于?y你怎么想？'],
    '?*x我?*z梦见?*y':['真的吗? --- ?y', '你在醒着的时候，以前想象过?y吗？', '你以前梦见过?y吗'],
    '?*x妈妈?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '她对你影响很大吗？'],
    '?*x爸爸?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '他对你影响很大吗？', '每当你想起你爸爸的时候， 你还会想起其他的吗?'],
    '?*x我愿意?*y': ['我可以帮你?y吗？', '你可以解释一下，为什么想?y'],
    '?*x我很难过，因为?*y': ['我听到你这么说， 也很难过', '?y不应该让你这么难过的'],
    '?*x难过?*y': ['我听到你这么说， 也很难过',
                 '不应该让你这么难过的，你觉得你拥有什么，就会不难过?',
                 '你觉得事情变成什么样，你就不难过了?'],
    '?*x就像?*y': ['你觉得?x和?y有什么相似性？', '?x和?y真的有关系吗？', '怎么说？'],
    '?*x和?*y都?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x和?*y一样?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x我是?*y': ['真的吗？', '?x想告诉你，或许我早就知道你是?y', '你为什么现在才告诉我你是?y'],
    '?*x我是?*y吗': ['如果你是?y会怎么样呢？', '你觉得你是?y吗', '如果你是?y，那一位着什么?'],
    '?*x你是?*y吗':  ['你为什么会对我是不是?y感兴趣?', '那你希望我是?y吗', '你要是喜欢， 我就会是?y'],
    '?*x你是?*y' : ['为什么你觉得我是?y'],
    '?*x因为?*y' : ['?y是真正的原因吗？', '你觉得会有其他原因吗?'],
    '?*x我不能?*y': ['你或许现在就能?*y', '如果你能?*y,会怎样呢？'],
    '?*x我觉得?*y': ['你经常这样感觉吗？', '除了到这个，你还有什么其他的感觉吗？'],
    '?*x我?*y你?*z': ['其实很有可能我们互相?y'],
    '?*x你为什么不?*y': ['你自己为什么不?y', '你觉得我不会?y', '等我心情好了，我就?y'],
    '?*x好的?*y': ['好的', '你是一个很正能量的人'],
    '?*x嗯嗯?*y': ['好的', '你是一个很正能量的人'],
    '?*x不嘛?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x不要?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x有些人?*y': ['具体是哪些人呢?'],
    '?*x有的人?*y': ['具体是哪些人呢?'],
    '?*x某些人?*y': ['具体是哪些人呢?'],
    '?*x每个人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x所有人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x总是?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x一直?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x或许?*y': ['你看起来不太确定'],
    '?*x可能?*y': ['你看起来不太确定'],
    '?*x他们是?*y吗？': ['你觉得他们可能不是?y？'],
    '?*x': ['很有趣', '请继续', '我不太确定我很理解你说的, 能稍微详细解释一下吗?']
}

In [50]:
import jieba
import re

In [51]:
def check_contain_chinese(check_str):

    for ch in check_str:
        if u'\u4e00' <= ch <= u'\u9fff':
            return True
    return False

In [52]:
check_contain_chinese('我喜欢?X')

True

### 怎么判断中文字符串中有无?X

In [67]:
if '?X' in u'我喜欢?X':
    print(True)

True


In [80]:
def is_variable(pat):
    sentense = 'u'+pat

    if '?X' in pat:
        print(True)

In [81]:
is_variable('我喜欢?X')

True
